In [36]:
from pathlib import Path
import os
import json
import tqdm

In [2]:
def rm_tree(pth):
    pth = Path(pth)
    for child in pth.glob('*'):
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

In [3]:
path_root = Path('./CIFAR10/resnet19/kaiming_uniform/tune_zoo_cifar10_resnet18_kaiming_uniform')

In [29]:
path_lst = [p.absolute() for p in path_root.iterdir() if p.is_dir()]

In [30]:
print(len(path_lst))

1000


# remove optimzier states

In [34]:
def remove_all_but_last_optimizer(pdx,keep_edx):
    for edx in range(0,51):
        if edx == keep_edx:
            continue
        optimizer_state_path = Path(pdx).joinpath(f'checkpoint_{edx:06d}','optimizer')
        try:
#             print(f'remove optimizer {optimizer_state_path}')
            if optimizer_state_path.is_file():
                optimizer_state_path.unlink()
#             rm_tree(optimizer_state_path)
        except Exception as e:
            print(e)

In [37]:
for pdx in tqdm.tqdm(path_lst):
    remove_all_but_last_optimizer(pdx,50)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [26:40<00:00,  1.60s/it]


# make copy with ~10%

In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from pathlib import Path
import os
import json
import shutil
import errno
import tqdm

# set environment variables to limit cpu usage
os.environ["OMP_NUM_THREADS"] = "4"  # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4"  # export OPENBLAS_NUM_THREADS=4
os.environ["MKL_NUM_THREADS"] = "6"  # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"  # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6"  # export NUMEXPR_NUM_THREADS=6

In [2]:


# %%
def copy_dir(src, dest):
    try:
        shutil.copytree(src, dest)
    except OSError as e:
        # If the error was caused because the source wasn't a directory
        if e.errno == errno.ENOTDIR:
            shutil.copy(src, dest)
        else:
            print("Directory not copied. Error: %s" % e)


def copy_file(src,dest):
    try:
        shutil.copyfile(src,dest)
    except Exception as e:
        print("File not copied. Error: %s" % e)
# %%


def move_dir(src, dest):
    # moves source directory recursively to destination
    try:
        shutil.move(src, dest)
    except OSError as e:
        print("Directory not copied. Error: %s" % e)





In [3]:
def get_root_list(root_dir, key=None):
    # generate root list
    # filter by key
    if key is not None:
        path_list_root = [
            pdx for pdx in Path(root_dir).iterdir() if pdx.is_dir() if key in str(pdx)
        ]
    else:
        path_list_root = [pdx for pdx in Path(root_dir).iterdir() if pdx.is_dir()]
    return path_list_root




In [4]:
def get_target_list(root_dir, target_dir, root_list):
    target_list = []
    for pdx in root_list:
        dest = Path(str(pdx).replace(root_dir, target_dir))
        target_list.append(dest)
    return target_list




In [66]:
def copy_selected_files(root_dir,target_dir,root_list):
    eedx_list = ["000001","000010","000020","000030","000040","000050"]

    # iterate over root list
    for root_ddx in tqdm.tqdm(root_list):
        # get list of files / dirs in current dir
        fpdx_list = [pdx for pdx in root_ddx.iterdir()]
        # iterate over files
        # make target directory
        dest_ddx = Path(str(root_ddx).replace(str(root_dir), str(target_dir)))
        dest_ddx.mkdir(exist_ok=True,parents=True)
        #
        for fpdx in fpdx_list:
            # if directory: check if relevant
            if fpdx.is_dir():
                if not str(fpdx)[-6:] in eedx_list:
                    continue
                    print(f'dir! {fpdx}')
            # if we get to this point, file / dir needs to be copied
            destdx = Path(str(fpdx).replace(str(root_dir), str(target_dir)))
            # copy 
            copy_dir(src=fpdx, dest=destdx)

In [54]:
#%%
path_root = Path('./CIFAR10/resnet19/kaiming_uniform/tune_zoo_cifar10_resnet18_kaiming_uniform')
path_dest = Path("./CIFAR10/resnet19/kaiming_uniform/cifar10_resnet18_kaiming_uniform_subset")

In [55]:
root_list = get_root_list(path_root)

In [68]:
copy_selected_files(root_dir=path_root,target_dir=path_dest,root_list=root_list)

 46%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                             | 457/1000 [48:18<1:04:48,  7.16s/it]

Directory not copied. Error: [('CIFAR10/resnet19/kaiming_uniform/tune_zoo_cifar10_resnet18_kaiming_uniform/NN_tune_trainable_0ecbc_00507_507_seed=508_2022-08-06_11-39-29/checkpoint_000030/.is_checkpoint', 'CIFAR10/resnet19/kaiming_uniform/cifar10_resnet18_kaiming_uniform_subset/NN_tune_trainable_0ecbc_00507_507_seed=508_2022-08-06_11-39-29/checkpoint_000030/.is_checkpoint', "[Errno 2] No such file or directory: 'CIFAR10/resnet19/kaiming_uniform/tune_zoo_cifar10_resnet18_kaiming_uniform/NN_tune_trainable_0ecbc_00507_507_seed=508_2022-08-06_11-39-29/checkpoint_000030/.is_checkpoint'")]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:45:38<00:00,  6.34s/it]


In [ ]:
# make zip archives

In [69]:
###Subset

In [1]:
from pathlib import Path
import os
import json
import tqdm

In [2]:
def rm_tree(pth):
    pth = Path(pth)
    for child in pth.glob('*'):
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

In [3]:
path_root = Path('./CIFAR10/resnet19/kaiming_uniform/cifar10_resnet18_kaiming_uniform_subset')

In [4]:
path_lst = [p.absolute() for p in path_root.iterdir() if p.is_dir()]

In [5]:
print(len(path_lst))

1000


In [76]:
def remove_all_but_last_optimizer(pdx,keep_edx):
    for edx in range(0,51):
        if edx == keep_edx:
            continue
        optimizer_state_path = Path(pdx).joinpath(f'checkpoint_{edx:06d}','optimizer')
        try:
#             print(f'remove optimizer {optimizer_state_path}')
            if optimizer_state_path.is_file():
                optimizer_state_path.unlink()
#             rm_tree(optimizer_state_path)
        except Exception as e:
            print(e)

In [77]:
for pdx in tqdm.tqdm(path_lst):
    remove_all_but_last_optimizer(pdx,51)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.41it/s]


In [6]:
def remove_checkoint(pdx,remove_edx):
    checkpoint_path = Path(pdx).joinpath(f'checkpoint_{remove_edx:06d}')
    rm_tree(checkpoint_path)

In [7]:
for pdx in tqdm.tqdm(path_lst):
    remove_checkoint(pdx,remove_edx=20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:01<00:00, 16.35it/s]
